<a href="https://colab.research.google.com/github/Hubert26/suicides_IPPAN/blob/main/suicides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Liblaries and settings

In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None

np.random.seed(42)

#Function definitions

#Data exploration

In [2]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/suicides_IPPAN/main/data/new_Suicides_2013-2022.csv', delimiter=',', low_memory=False)

In [3]:
df_raw

,suicide_id,report_date_year,report_date_month,age_group,gender,marital_status,education,work_study_info,livelihood_source,is_fatal,location_of_attack,method_of_committing,reason_of_attack,reason_of_attack_2,reason_of_attack_3,reason_of_attack_4,consciousness_state,substance_info,treatment_info
0,114 581 552,2013,1,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,114 584 284,2013,1,13-18,male,NaN,NaN,Student,NaN,1.0,Road/street/sidewalk,Throwing oneself under a moving vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,114 584 293,2013,1,13-18,female,Single,Primary education,Student,Dependent on another person,1.0,Other,Jumping from a height,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,114 584 312,2013,1,19-24,male,Single,NaN,NaN,NaN,1.0,NaN,Other,Love affair,NaN,NaN,NaN,NaN,NaN,NaN
4,114 584 399,2013,1,40-44,male,Single,NaN,Unemployed,NaN,1.0,Outbuildings,Hanging,NaN,NaN,NaN,NaN,Under the influence of alcohol,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113191,138 436 326,2022,12,50-54,male,Married,NaN,Part-time employment,No regular source of maintenance,1.0,"Basement, attic, garage corridor",Gas/pollutant poisoning,Family misunderstanding/family violence,NaN,NaN,NaN,NaN,NaN,NaN
113192,138 525 190,2022,8,50-54,male,Divorced,Vocational education,Unemployed,No regular source of maintenance,0.0,Other,Superficial self-mutilation,Other unspecified,NaN,NaN,NaN,Under the influence of alcohol,Alcohol abuse,NaN
113193,138 525 191,2022,11,40-44,male,Single,NaN,Part-time employment,No regular source of maintenance,0.0,Apartment/house,Ingesting other drugs,Love affair,NaN,NaN,NaN,Under the influence of alcohol,Alcohol abuse,NaN
113194,138 530 178,2022,11,45-49,male,NaN,NaN,NaN,NaN,0.0,Police facility,Strangulation,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_data = df_raw.copy()

In [5]:
reason_of_attack_encoded = pd.get_dummies(df_data['reason_of_attack'], columns=['reason_of_attack'])
reason_of_attack_encoded_column_names = reason_of_attack_encoded.columns.tolist()

In [23]:
set(df_data['reason_of_attack_2'].tolist())

{' ',
 'Bullying, cyberbullying, harassment',
 'Chronic illness',
 'Committing a crime or misdemeanor',
 'Conflict with non-family members',
 'Death of a close person',
 'Deterioration or sudden loss of health',
 'Family misunderstanding/family violence',
 'HIV carrier, AIDS patient',
 'Love affair',
 'Mental illness/mental disorders',
 'Other unspecified',
 'Permanent disability',
 'Physical illness',
 'Poor economic conditions/debt',
 'School or work problems',
 'Sudden loss of livelihood',
 'Threat or loss of housing',
 'Unwanted pregnancy',
 nan}

In [22]:
suicide_reason_map = {
    '0': np.nan,
    '1': "Love affair",
    '2': "Mental illness/mental disorders",
    '3': "Family misunderstanding/family violence",
    '4': "HIV carrier, AIDS patient",
    '5': "Sudden loss of livelihood",
    '6': "Poor economic conditions/debt",
    '7': "Chronic illness",
    '8': "School or work problems",
    '9': "Death of a close person",
    '10': "Committing a crime or misdemeanor",
    '11': "Permanent disability",
    '12': "Unwanted pregnancy",
    '13': "Physical illness",
    '14': "Deterioration or sudden loss of health",
    '15': "Conflict with non-family members",
    '16': "Threat or loss of housing",
    '17': "Bullying, cyberbullying, harassment",
    '18': "Other unspecified"
}

# Zastąpienie wartości w kolumnie 'cause_of_suicide' za pomocą słownika
df_data['reason_of_attack_2'] = df_data['reason_of_attack_2'].replace(suicide_reason_map)

In [18]:
set(df_data['reason_of_attack_3'].tolist())

{'Bullying, cyberbullying, harassment',
 'Chronic illness',
 'Committing a crime or misdemeanor',
 'Conflict with non-family members',
 'Death of a close person',
 'Deterioration or sudden loss of health',
 'Family misunderstanding/family violence',
 'HIV carrier, AIDS patient',
 'Love affair',
 'Mental illness/mental disorders',
 'Other unspecified',
 'Permanent disability',
 'Physical illness',
 'Poor economic conditions/debt',
 'School or work problems',
 'Sudden loss of livelihood',
 'Threat or loss of housing',
 nan}

In [19]:
set(df_data['reason_of_attack_4'].tolist())

{'Death of a close person',
 'Deterioration or sudden loss of health',
 'Love affair',
 'Mental illness/mental disorders',
 'Permanent disability',
 'Physical illness',
 'Poor economic conditions/debt',
 'School or work problems',
 'Threat or loss of housing',
 nan}

In [38]:
# Utworzenie listy unikalnych wartości
reasons_of_attack_values = df_data['reason_of_attack'].tolist() + \
                           df_data['reason_of_attack_2'].tolist() + \
                           df_data['reason_of_attack_3'].tolist() + \
                           df_data['reason_of_attack_4'].tolist()

# Usunięcie wartości pustych i NaN oraz posortowanie
reasons_of_attack_values = list(set(reasons_of_attack_values))



In [45]:
reasons_of_attack_values = [value for value in reasons_of_attack_values if value != ' ']

In [47]:
reasons_of_attack_values.pop(0)

nan

In [48]:
reasons_of_attack_values

['Physical illness',
 'Poor economic conditions/debt',
 'Love affair',
 'Permanent disability',
 'Chronic illness',
 'Family misunderstanding/family violence',
 'Committing a crime or misdemeanor',
 'Deterioration or sudden loss of health',
 'Bullying, cyberbullying, harassment',
 'School or work problems',
 'Sudden loss of livelihood',
 'Conflict with non-family members',
 'HIV carrier, AIDS patient',
 'Other unspecified',
 'Unwanted pregnancy',
 'Mental illness/mental disorders',
 'Death of a close person',
 'Threat or loss of housing']

In [49]:
len(reasons_of_attack_values)

18

In [50]:
len(reason_of_attack_encoded_column_names)

18

In [6]:
reason_of_attack_encoded_column_names

['Bullying, cyberbullying, harassment',
 'Chronic illness',
 'Committing a crime or misdemeanor',
 'Conflict with non-family members',
 'Death of a close person',
 'Deterioration or sudden loss of health',
 'Family misunderstanding/family violence',
 'HIV carrier, AIDS patient',
 'Love affair',
 'Mental illness/mental disorders',
 'Other unspecified',
 'Permanent disability',
 'Physical illness',
 'Poor economic conditions/debt',
 'School or work problems',
 'Sudden loss of livelihood',
 'Threat or loss of housing',
 'Unwanted pregnancy']

In [51]:
mask_reason_of_attack_1 = pd.DataFrame(columns=reason_of_attack_encoded_column_names)
for i in range(len(reason_of_attack_encoded_column_names)):
    mask_reason_of_attack_1[reason_of_attack_encoded_column_names[i]] = (df_data['reason_of_attack'] == reason_of_attack_encoded_column_names[i])

In [52]:
mask_reason_of_attack_2 = pd.DataFrame(columns=reason_of_attack_encoded_column_names)
for i in range(len(reason_of_attack_encoded_column_names)):
    mask_reason_of_attack_2[reason_of_attack_encoded_column_names[i]] = (df_data['reason_of_attack_2'] == reason_of_attack_encoded_column_names[i])

In [53]:
mask_reason_of_attack_3 = pd.DataFrame(columns=reason_of_attack_encoded_column_names)
for i in range(len(reason_of_attack_encoded_column_names)):
    mask_reason_of_attack_3[reason_of_attack_encoded_column_names[i]] = (df_data['reason_of_attack_3'] == reason_of_attack_encoded_column_names[i])

In [54]:
mask_reason_of_attack_4 = pd.DataFrame(columns=reason_of_attack_encoded_column_names)
for i in range(len(reason_of_attack_encoded_column_names)):
    mask_reason_of_attack_4[reason_of_attack_encoded_column_names[i]] = (df_data['reason_of_attack_4'] == reason_of_attack_encoded_column_names[i])

In [55]:
mask_reason_of_attack = mask_reason_of_attack_1 | mask_reason_of_attack_2 | mask_reason_of_attack_3 | mask_reason_of_attack_4

In [57]:
mask_reason_of_attack.columns = ["reasons_of_attack_" + column_name for column_name in reason_of_attack_encoded_column_names]

In [58]:
mask_reason_of_attack = mask_reason_of_attack.astype(int)

In [59]:
mask_reason_of_attack

,"reasons_of_attack_Bullying, cyberbullying, harassment",reasons_of_attack_Chronic illness,reasons_of_attack_Committing a crime or misdemeanor,reasons_of_attack_Conflict with non-family members,reasons_of_attack_Death of a close person,reasons_of_attack_Deterioration or sudden loss of health,reasons_of_attack_Family misunderstanding/family violence,"reasons_of_attack_HIV carrier, AIDS patient",reasons_of_attack_Love affair,reasons_of_attack_Mental illness/mental disorders,reasons_of_attack_Other unspecified,reasons_of_attack_Permanent disability,reasons_of_attack_Physical illness,reasons_of_attack_Poor economic conditions/debt,reasons_of_attack_School or work problems,reasons_of_attack_Sudden loss of livelihood,reasons_of_attack_Threat or loss of housing,reasons_of_attack_Unwanted pregnancy
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113191,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
113192,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
113193,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
113194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
df_data = pd.concat([df_data, mask_reason_of_attack], axis=1)


In [62]:
mask_reason_of_attack["reason_of_attack_count"] = mask_reason_of_attack.sum(axis=1)

In [63]:

df_data = pd.concat([df_data, mask_reason_of_attack["reason_of_attack_count"]], axis=1)

In [65]:
columns_to_drop = ['reason_of_attack', 'reason_of_attack_2', 'reason_of_attack_3', 'reason_of_attack_4']
df_data = df_data.drop(columns=columns_to_drop)


In [66]:
df_data

,suicide_id,report_date_year,report_date_month,age_group,gender,marital_status,education,work_study_info,livelihood_source,is_fatal,location_of_attack,method_of_committing,consciousness_state,substance_info,treatment_info,"reasons_of_attack_Bullying, cyberbullying, harassment",reasons_of_attack_Chronic illness,reasons_of_attack_Committing a crime or misdemeanor,reasons_of_attack_Conflict with non-family members,reasons_of_attack_Death of a close person,reasons_of_attack_Deterioration or sudden loss of health,reasons_of_attack_Family misunderstanding/family violence,"reasons_of_attack_HIV carrier, AIDS patient",reasons_of_attack_Love affair,reasons_of_attack_Mental illness/mental disorders,reasons_of_attack_Other unspecified,reasons_of_attack_Permanent disability,reasons_of_attack_Physical illness,reasons_of_attack_Poor economic conditions/debt,reasons_of_attack_School or work problems,reasons_of_attack_Sudden loss of livelihood,reasons_of_attack_Threat or loss of housing,reasons_of_attack_Unwanted pregnancy,reason_of_attack_count
0,114 581 552,2013,1,NaN,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,114 584 284,2013,1,13-18,male,NaN,NaN,Student,NaN,1.0,Road/street/sidewalk,Throwing oneself under a moving vehicle,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,114 584 293,2013,1,13-18,female,Single,Primary education,Student,Dependent on another person,1.0,Other,Jumping from a height,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,114 584 312,2013,1,19-24,male,Single,NaN,NaN,NaN,1.0,NaN,Other,NaN,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,114 584 399,2013,1,40-44,male,Single,NaN,Unemployed,NaN,1.0,Outbuildings,Hanging,Under the influence of alcohol,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113191,138 436 326,2022,12,50-54,male,Married,NaN,Part-time employment,No regular source of maintenance,1.0,"Basement, attic, garage corridor",Gas/pollutant poisoning,NaN,NaN,NaN,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
113192,138 525 190,2022,8,50-54,male,Divorced,Vocational education,Unemployed,No regular source of maintenance,0.0,Other,Superficial self-mutilation,Under the influence of alcohol,Alcohol abuse,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
113193,138 525 191,2022,11,40-44,male,Single,NaN,Part-time employment,No regular source of maintenance,0.0,Apartment/house,Ingesting other drugs,Under the influence of alcohol,Alcohol abuse,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
113194,138 530 178,2022,11,45-49,male,NaN,NaN,NaN,NaN,0.0,Police facility,Strangulation,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
from google.colab import drive

# Montowanie dysku Google
drive.mount('/content/drive')

# Ścieżka docelowa na dysku Google
file_path_drive = '/content/drive/My Drive/df_data.csv'

# Zapisanie ramki danych do pliku CSV na dysku Google
df_data.to_csv(file_path_drive, index=False)

Mounted at /content/drive


##NaN exploration in variables

In [16]:
df_data.shape

(113196, 19)

In [17]:
df_data[df_data['suicide_id'].duplicated()].suicide_id #Sprawdzenie duplikatów

Series([], Name: suicide_id, dtype: object)

In [18]:
##Brakujące dane
total = df_data.isnull().sum().sort_values(ascending=False)
percent = 100*(df_data.isnull().sum()/df_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [19]:
missing_data

,Total,Percent
reason_of_attack_4,113107,99.921375
reason_of_attack_3,112405,99.301212
treatment_info,108281,95.657974
reason_of_attack_2,102499,90.550019
substance_info,93753,82.823598
education,77929,68.844305
consciousness_state,58827,51.969151
work_study_info,55930,49.409873
livelihood_source,49338,43.586346
reason_of_attack,40204,35.517156


##NaN exploration in rows